In [1]:
from RANGE_go.ga_abc import GA_ABC
from RANGE_go.cluster_model import cluster_model
from RANGE_go.energy_calculation import energy_computation

import numpy as np
#import matplotlib.pyplot as plt
import os

from ase import Atoms
from ase.constraints import FixAtoms
from ase.visualize import view
from ase.db import connect
from ase.io import read, write

from ase.constraints import FixBondLengths, FixAtoms

import nglview as nv

# Environment setup
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [2]:
# Proper calculation methods
from mace.calculators import mace_mp
from xtb.ase.calculator import XTB


C:\MyApp\Anaconda\envs\RDkit_from_globopt\Lib\site-packages\e3nn\o3\_wigner.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  _Jd, _W3j_flat, _W3j_indices = torch.load(os

cuequivariance or cuequivariance_torch is not available. Cuequivariance acceleration will be disabled.


In [3]:
# Helper if we want to use MLIP
Potential_test = 'mace-mpa-0-medium.model'

# Example 1: A simple water cluster
"inbox box" constraint

In [4]:
water = "xyz_structures/Water.xyz"

# System: a cluster of 10 water molecules
input_molecules = [water]
input_num_of_molecules = [ 10] 

# Put 10 water molecules in a box region where lower limit is 0 and upper limit is 6
input_constraint_type = [ 'in_box' ]
input_constraint_value = [ (0,0,0,6,6,6) ]

# Generate modeling setting
cluster = cluster_model(input_molecules, input_num_of_molecules, input_constraint_type, input_constraint_value)
cluster_template, cluster_boundary, cluster_conversion_rule = cluster.generate_bounds()  

# Assign coarse calculator
coarse_opt_parameter = dict(coarse_calc_eps='UFF', coarse_calc_sig='UFF', coarse_calc_chg=0, 
                            coarse_calc_step=30, coarse_calc_fmax=10, coarse_calc_constraint=None)

# Assign fine calculator, e.g. MLIP
#ase_calculator = mace_mp(model=Potential_test, dispersion=True, default_dtype="float64", device='cuda')
# Or semi-empirical xTB via Python interface
ase_calculator = XTB(method="GFN2-xTB") 
geo_opt_parameter = dict(fmax=0.1, steps=300, ase_constraint=None) # fmax here is large to make the run faster for testing purpose. Need to lower it in practice

# Set calculation parameters
computation = energy_computation(templates = cluster_template, go_conversion_rule = cluster_conversion_rule,   
                                 calculator = ase_calculator, 
                                 geo_opt_para = geo_opt_parameter, 
                                 calculator_type = 'ase',    
                                 if_coarse_calc = True, coarse_calc_para = coarse_opt_parameter,
                                )

# Set the optimization parameters. Just a short test here.
optimization = GA_ABC(computation.obj_func_compute_energy, cluster_boundary,
                      colony_size=10, limit=40, 
                      max_iteration=3, initial_population_scaler=2,
                      ga_interval=1, ga_parents=5, mutate_rate=0.5, mutat_sigma=0.05,
                      output_database = 'test-1.db'
                      # Restart option
                      #restart_from_pool = 'structure_pool.db',
                      )

In [5]:
# Now run the search. Output structures are written to 'structure_pool.db' by default
optimization.run(print_interval=1, if_return_results=False)

Setup ready...

        
     /\       _     _
    /  \     / \   / \       _ 
   /    \   /   \_/   \     / \   _
  /      \_/           \   /   \_/ \
 /                      \_/         \
/                                    \
████     ▄█▄    █   █   ███    ████   \
█   █    █ █    ██  █  █      █        \
█▄▄██   ██▄██   █ █ █  █  ██  ████      \
█  █    █   █   █  ██  █   █  █          \
█   █  █     █  █   █   ████   ████       \
                                           \___
----------------------------------------------->>
                                           
RANGE: a Robust Adaptive Nature-inspired Global Explorer of potential energy surfaces

Please cite: Difan Zhang, Małgorzata Z. Makoś, Roger Rousseau, Vassiliki-Alexandra Glezakou; RANGE: A robust adaptive nature-inspired global explorer of potential energy surfaces. J. Chem. Phys. 21 October 2025; 163 (15): 152501. https://doi.org/10.1063/5.0288910

For any questions, please contact us:
    Difan Zhang, Oak Ridge Na

C:\Users\d2j\OneDrive - Oak Ridge National Laboratory\Research_folder\Project_002_Glob_optimizer\RANGE\RANGE_go\energy_calculation.py:504: UserWarning: Gimbal lock detected. Setting third angle to zero since it is not possible to uniquely determine all angles.
  vec = self.cluster_to_vector( atoms, vec ) # update vec since we optimized the structure


Time cost (s) for  compute_000024_round_1_ol_1_from_2_8_4_1               15.668  with energy:  -1383.19437707
Time cost (s) for  compute_000025_round_1_ol_2_from_9_1_4_3                5.111  with energy:  -1383.05215818
Time cost (s) for  compute_000026_round_1_ol_3_from_5_4_1_3                9.421  with energy:  -1382.72578967
Time cost (s) for  compute_000027_round_1_ga_0_from_6_GM                   3.282  with energy:  -1383.42691568
Dynamic info at End of Iteration     1: EM_num=1 OL_num=  4 SC_limit= 35 performed to find best_Y=         -1383.4 Life=    7 Total_size=   27 Generate_size=   27 with current Ratio=0.26
Time cost (s) for  compute_000028_round_2_em_0_from_2_3_1                   10.4  with energy:  -1382.89274635
Time cost (s) for  compute_000029_round_2_ga_0_from_7_0                    7.167  with energy:  -1382.86558327
Time cost (s) for  compute_000030_round_2_ol_0_from_1_0_4_6               10.097  with energy:  -1382.60840021
Time cost (s) for  compute_000031_ro

In [5]:
db = connect('test-1.db')
traj = [row.toatoms() for row in db.select()]
view = nv.show_asetraj(traj)
view

NGLWidget(max_frame=39)

In [ ]:
# Note: for larger systems, we can use the double-optimization method to optimize geometries with constrains after coarse opt, before fine opt.
bond_pairs = cluster.compute_system_bond_pair()
ase_constraint = FixBondLengths( bond_pairs )
geo_opt_parameter = dict(fmax=2, steps=200, ase_constraint=ase_constraint, Dual_stage_optimization=dict(fmax=0.05, steps=200) )

# Example 2: Gas adsorption on a cluster surface
"at_position" + "on_surface" constraint

In [4]:
copper_12 = 'xyz_structures/Cu12.xyz'
methane = 'xyz_structures/Methane.xyz'
water = "xyz_structures/Water.xyz"

# System: Put 1 CH4 and 1 H2O on Cu12 surface
input_molecules = [copper_12, methane, water]
input_num_of_molecules = [1, 1, 1] 

# Put Cu12 at a fixed position, and put CH4 and H2O on its surface. Initial surface distance 1.9-2.1
input_constraint_type = [ 'at_position', 'on_surface', 'on_surface' ]
input_constraint_value = [ (0,0,0), (0, (1.9, 2.1), 1,0) , (0, (1.9, 2.1), 0,1) ]

# ---------- Similar settings ---------- #
cluster = cluster_model(input_molecules, input_num_of_molecules, input_constraint_type, input_constraint_value)
cluster_template, cluster_boundary, cluster_conversion_rule = cluster.generate_bounds() 

coarse_opt_parameter = dict(coarse_calc_eps='UFF', coarse_calc_sig='UFF', coarse_calc_chg=0, 
                            coarse_calc_step=30, coarse_calc_fmax=10, coarse_calc_constraint=None)
# Use MLIP from MACE + D3 dispersion
ase_calculator = mace_mp(model=Potential_test, dispersion=True, default_dtype="float64", device='cuda')
geo_opt_parameter = dict(fmax=0.1, steps=300, ase_constraint=None)

computation = energy_computation(templates = cluster_template, go_conversion_rule = cluster_conversion_rule,   
                                 calculator = ase_calculator, 
                                 geo_opt_para = geo_opt_parameter, 
                                 calculator_type = 'ase',    
                                 if_coarse_calc = True, coarse_calc_para = coarse_opt_parameter,
                                 save_output_level = 'Full',  # Write all the output files to output if "Full"
                                )

optimization = GA_ABC(computation.obj_func_compute_energy, cluster_boundary,
                      colony_size=10, limit=40, 
                      max_iteration=3, initial_population_scaler=2,
                      ga_interval=1, ga_parents=5, mutate_rate=0.5, mutat_sigma=0.05,
                      output_directory = 'results-2',
                      output_database = 'test-2.db'
                      )

Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.


C:\MyApp\Anaconda\envs\RDkit_from_globopt\Lib\site-packages\mace\calculators\mace.py:197: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, map_location

Using head default out of ['default']
Using TorchDFTD3Calculator for D3 dispersion corrections


In [5]:
optimization.run(print_interval=1, if_return_results=False)

Setup ready...

        
     /\       _     _
    /  \     / \   / \       _ 
   /    \   /   \_/   \     / \   _
  /      \_/           \   /   \_/ \
 /                      \_/         \
/                                    \
████     ▄█▄    █   █   ███    ████   \
█   █    █ █    ██  █  █      █        \
█▄▄██   ██▄██   █ █ █  █  ██  ████      \
█  █    █   █   █  ██  █   █  █          \
█   █  █     █  █   █   ████   ████       \
                                           \___
----------------------------------------------->>
                                           
RANGE: a Robust Adaptive Nature-inspired Global Explorer of potential energy surfaces

Please cite: Difan Zhang, Małgorzata Z. Makoś, Roger Rousseau, Vassiliki-Alexandra Glezakou; RANGE: A robust adaptive nature-inspired global explorer of potential energy surfaces. J. Chem. Phys. 21 October 2025; 163 (15): 152501. https://doi.org/10.1063/5.0288910

For any questions, please contact us:
    Difan Zhang, Oak Ridge Na

C:\Users\d2j\OneDrive - Oak Ridge National Laboratory\Research_folder\Project_002_Glob_optimizer\RANGE\RANGE_go\energy_calculation.py:504: UserWarning: Gimbal lock detected. Setting third angle to zero since it is not possible to uniquely determine all angles.
  vec = self.cluster_to_vector( atoms, vec ) # update vec since we optimized the structure


Time cost (s) for  compute_000035_round_3_ga_0_from_3_5                   20.407  with energy:  -74.49623354
Time cost (s) for  compute_000036_round_3_ga_1_from_2_0                    6.898  with energy:  -74.2177112
Time cost (s) for  compute_000037_round_3_ol_0_from_6_9_3_0               23.467  with energy:  -74.84400256
Time cost (s) for  compute_000038_round_3_ol_1_from_5_2_7_8               13.229  with energy:  -74.25439301
Time cost (s) for  compute_000039_round_3_ga_0_from_1_GM                   3.659  with energy:  -73.3633742
Time cost (s) for  compute_000040_round_3_ga_1_from_8_GM                    4.76  with energy:  -74.22649863
Dynamic info at End of Iteration     3: EM_num=1 OL_num=  2 SC_limit= 30 performed to find best_Y=        -74.8529 Life=   20 Total_size=   40 Generate_size=   40 with current Ratio=0.5
Job completed with best GM: -74.8528780715121 at compute_000001_round_0_sc_0 that has survived last 20 times of 40 generations


In [7]:
db = connect('test-2.db')
traj = [row.toatoms() for row in db.select()][:]
view = nv.show_asetraj(traj)
view

NGLWidget(max_frame=76)

# Example 3: A core-shell particle
"in_sphere_shell" constraint

In [10]:
Cu = 'xyz_structures/Cu.xyz'
Zn = 'xyz_structures/Zn.xyz'

# System: Put 15 Cu and 30 Zn as a Cu-core/Zn-shell particle
input_molecules = [Cu, Zn]
input_num_of_molecules = [15, 30] 

# Spherical region is centered at (0,0,0), with Cu in R<3 and Zn in 3<R<5
input_constraint_type = [ 'in_sphere_shell', 'in_sphere_shell']
input_constraint_value = [ (0,0,0, 3,3,3, 1), (0,0,0, 5,5,5, 0.4)]

# ---------- Similar settings ---------- #
cluster = cluster_model(input_molecules, input_num_of_molecules, input_constraint_type, input_constraint_value)
cluster_template, cluster_boundary, cluster_conversion_rule = cluster.generate_bounds() 

coarse_opt_parameter = dict(coarse_calc_eps='UFF', coarse_calc_sig='UFF', coarse_calc_chg=0, 
                            coarse_calc_step=30, coarse_calc_fmax=10, coarse_calc_constraint=None)
# Use MLIP from MACE + D3 dispersion
ase_calculator = mace_mp(model=Potential_test, dispersion=True, default_dtype="float64", device='cuda')
geo_opt_parameter = dict(fmax=0.1, steps=300, ase_constraint=None)

computation = energy_computation(templates = cluster_template, go_conversion_rule = cluster_conversion_rule,   
                                 calculator = ase_calculator, 
                                 geo_opt_para = geo_opt_parameter, 
                                 calculator_type = 'ase',    
                                 if_coarse_calc = True, coarse_calc_para = coarse_opt_parameter,
                                 save_output_level = 'Simple',  
                                )

optimization = GA_ABC(computation.obj_func_compute_energy, cluster_boundary,
                      colony_size=10, limit=40, 
                      max_iteration=3, initial_population_scaler=2,
                      ga_interval=1, ga_parents=5, mutate_rate=0.5, mutat_sigma=0.05,
                      output_directory = 'results-3',
                      output_database = 'test-3.db'
                      )

Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.


C:\MyApp\Anaconda\envs\RDkit_from_globopt\Lib\site-packages\mace\calculators\mace.py:197: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, map_location

Using head default out of ['default']
Using TorchDFTD3Calculator for D3 dispersion corrections


In [11]:
optimization.run(print_interval=1, if_return_results=False)

Setup ready...

        
     /\       _     _
    /  \     / \   / \       _ 
   /    \   /   \_/   \     / \   _
  /      \_/           \   /   \_/ \
 /                      \_/         \
/                                    \
████     ▄█▄    █   █   ███    ████   \
█   █    █ █    ██  █  █      █        \
█▄▄██   ██▄██   █ █ █  █  ██  ████      \
█  █    █   █   █  ██  █   █  █          \
█   █  █     █  █   █   ████   ████       \
                                           \___
----------------------------------------------->>
                                           
RANGE: a Robust Adaptive Nature-inspired Global Explorer of potential energy surfaces

Please cite: Difan Zhang, Małgorzata Z. Makoś, Roger Rousseau, Vassiliki-Alexandra Glezakou; RANGE: A robust adaptive nature-inspired global explorer of potential energy surfaces. J. Chem. Phys. 21 October 2025; 163 (15): 152501. https://doi.org/10.1063/5.0288910

For any questions, please contact us:
    Difan Zhang, Oak Ridge Na

C:\Users\d2j\OneDrive - Oak Ridge National Laboratory\Research_folder\Project_002_Glob_optimizer\RANGE\RANGE_go\energy_calculation.py:504: UserWarning: Gimbal lock detected. Setting third angle to zero since it is not possible to uniquely determine all angles.
  vec = self.cluster_to_vector( atoms, vec ) # update vec since we optimized the structure
C:\Users\d2j\OneDrive - Oak Ridge National Laboratory\Research_folder\Project_002_Glob_optimizer\RANGE\RANGE_go\energy_calculation.py:540: UserWarning: Gimbal lock detected. Setting third angle to zero since it is not possible to uniquely determine all angles.
  vec = self.cluster_to_vector( atoms, vec )    # Update vec again after opt


Time cost (s) for  compute_000001_round_0_sc_0                            32.766  with energy:  -87.69894068
Time cost (s) for  compute_000002_round_0_sc_1                            31.333  with energy:  -86.59843145
Time cost (s) for  compute_000003_round_0_sc_2                            32.308  with energy:  -87.5573353
Time cost (s) for  compute_000004_round_0_sc_3                            31.596  with energy:  -86.36746934
Time cost (s) for  compute_000005_round_0_sc_4                            28.797  with energy:  -85.81499939
Time cost (s) for  compute_000006_round_0_sc_5                             42.06  with energy:  -87.17904215
Time cost (s) for  compute_000007_round_0_sc_6                            24.666  with energy:  -85.64795636
Time cost (s) for  compute_000008_round_0_sc_7                            30.527  with energy:  -87.41331567
Time cost (s) for  compute_000009_round_0_sc_8                            37.011  with energy:  -87.03080795
Time cost (s) for  c

In [4]:
db = connect('test-3.db')
atoms = db.get_atoms(id=1) 
view( atoms, viewer='x3d' )


# Example 4: A supported particle on surface
"at_position" and "in_box" constraint

In [4]:
substrate = 'xyz_structures/gC3N4-layer.xyz'
Pt = Atoms('Pt', positions=[(0, 0, 0)])

# System: Put 4 Pt on top of substrate surface
input_molecules = [substrate, Pt] 
input_num_of_molecules = [1, 4] 

# For at_position, when no parameter is provided, use it as-is.
input_constraint_type = ['at_position', 'in_box']
input_constraint_value = [(), (7.17, 12.5, 14.5, 14.3, 17.9, 18) ]

# ---------- Similar settings, but with PBC now---------- #
cluster = cluster_model(input_molecules, input_num_of_molecules, input_constraint_type, input_constraint_value,
                        pbc_box=(21.404922, 24.706836, 30), # For substrate slab
                       )
cluster_template, cluster_boundary, cluster_conversion_rule = cluster.generate_bounds() 

coarse_opt_parameter = dict(coarse_calc_eps='UFF', coarse_calc_sig='UFF', coarse_calc_chg=0, 
                            coarse_calc_step=30, coarse_calc_fmax=10, coarse_calc_constraint=None)
# Consider atom constraints to reduce unnecessary cost
ase_constraint = FixAtoms(indices=[at.index for at in cluster.system_atoms if at.symbol in ['C'] ])
ase_calculator = mace_mp(model=Potential_test, dispersion=True, default_dtype="float64", device='cuda')
geo_opt_parameter = dict(fmax=0.1, steps=100, ase_constraint=ase_constraint)

computation = energy_computation(templates = cluster_template, go_conversion_rule = cluster_conversion_rule,   
                                 calculator = ase_calculator, 
                                 geo_opt_para = geo_opt_parameter, 
                                 calculator_type = 'ase',    
                                 if_coarse_calc = True, coarse_calc_para = coarse_opt_parameter,
                                 save_output_level = 'Simple',  
                                )

optimization = GA_ABC(computation.obj_func_compute_energy, cluster_boundary,
                      colony_size=10, limit=40, 
                      max_iteration=3, initial_population_scaler=2,
                      ga_interval=1, ga_parents=5, mutate_rate=0.5, mutat_sigma=0.05,
                      output_directory = 'results-4',
                      output_database = 'test-4.db'
                      )

Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.


C:\MyApp\Anaconda\envs\RDkit_from_globopt\Lib\site-packages\mace\calculators\mace.py:197: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, map_location

Using head default out of ['default']
Using TorchDFTD3Calculator for D3 dispersion corrections


In [5]:
optimization.run(print_interval=1, if_return_results=False)

Setup ready...

        
     /\       _     _
    /  \     / \   / \       _ 
   /    \   /   \_/   \     / \   _
  /      \_/           \   /   \_/ \
 /                      \_/         \
/                                    \
████     ▄█▄    █   █   ███    ████   \
█   █    █ █    ██  █  █      █        \
█▄▄██   ██▄██   █ █ █  █  ██  ████      \
█  █    █   █   █  ██  █   █  █          \
█   █  █     █  █   █   ████   ████       \
                                           \___
----------------------------------------------->>
                                           
RANGE: a Robust Adaptive Nature-inspired Global Explorer of potential energy surfaces

Please cite: Difan Zhang, Małgorzata Z. Makoś, Roger Rousseau, Vassiliki-Alexandra Glezakou; RANGE: A robust adaptive nature-inspired global explorer of potential energy surfaces. J. Chem. Phys. 21 October 2025; 163 (15): 152501. https://doi.org/10.1063/5.0288910

For any questions, please contact us:
    Difan Zhang, Oak Ridge Na

C:\Users\d2j\OneDrive - Oak Ridge National Laboratory\Research_folder\Project_002_Glob_optimizer\RANGE\RANGE_go\energy_calculation.py:504: UserWarning: Gimbal lock detected. Setting third angle to zero since it is not possible to uniquely determine all angles.
  vec = self.cluster_to_vector( atoms, vec ) # update vec since we optimized the structure
C:\Users\d2j\OneDrive - Oak Ridge National Laboratory\Research_folder\Project_002_Glob_optimizer\RANGE\RANGE_go\energy_calculation.py:540: UserWarning: Gimbal lock detected. Setting third angle to zero since it is not possible to uniquely determine all angles.
  vec = self.cluster_to_vector( atoms, vec )    # Update vec again after opt


Time cost (s) for  compute_000001_round_0_sc_0                           345.358  with energy:  -1438.29473522
Time cost (s) for  compute_000002_round_0_sc_1                           203.805  with energy:  -1434.08768988
Time cost (s) for  compute_000003_round_0_sc_2                           355.618  with energy:  -1437.48414112
Time cost (s) for  compute_000004_round_0_sc_3                           196.295  with energy:  -1437.05642867
Time cost (s) for  compute_000005_round_0_sc_4                           195.885  with energy:  -1435.73729414
Time cost (s) for  compute_000006_round_0_sc_5                           317.563  with energy:  -1437.80394372
Time cost (s) for  compute_000007_round_0_sc_6                           225.495  with energy:  -1437.53257412
Time cost (s) for  compute_000008_round_0_sc_7                            196.44  with energy:  -1437.86107852
Time cost (s) for  compute_000009_round_0_sc_8                           276.498  with energy:  -1437.17562885
T

In [12]:
db = connect('test-4.db')
atoms = db.get_atoms(id=38) 
view( atoms, viewer='x3d' )


# Example 5: Binding inside a porous framework
"at_position" and "in_pore" constraint

In [5]:
zeolite = 'xyz_structures/Zeolite.xyz'
water = 'xyz_structures/Water.xyz'
hydronium = 'xyz_structures/Water_H+.xyz'

input_molecules = [zeolite, water, hydronium]
input_num_of_molecules = [1, 3, 1]

# Put water and hydronium inside a pore space defined by atoms in zeolite
input_constraint_type = ['at_position', 'in_pore', 'in_pore']
pore_atom_indices = (287,95,96,97,113,106,250,265,270,136,262,258,261)
input_constraint_value = [ (), (0, pore_atom_indices, 0.2), (0, pore_atom_indices, 0.2) ]

# ---------- Similar settings, but with PBC now---------- #
cluster = cluster_model(input_molecules, input_num_of_molecules, input_constraint_type, input_constraint_value,
                        pbc_box=( 20.022, 19.899, 13.383 ) ) # PBC system for zeolite
cluster_template, cluster_boundary, cluster_conversion_rule = cluster.generate_bounds() 

coarse_opt_parameter = dict(coarse_calc_eps='UFF', coarse_calc_sig='UFF', coarse_calc_chg=0, 
                            coarse_calc_step=30, coarse_calc_fmax=10, coarse_calc_constraint=None)
# Use a dual optimziation method to reduct computational cost
bond_pairs = cluster.compute_system_bond_pair()
bond_constraint = FixBondLengths( bond_pairs )
atom_constraint = FixAtoms(indices=[at.index for at in cluster.system_atoms if at.symbol in ['Si'] ])
ase_constraint = [atom_constraint, bond_constraint]
geo_opt_parameter = dict(fmax=2, steps=100, ase_constraint=ase_constraint, Dual_stage_optimization=dict(fmax=0.1, steps=100) )
ase_calculator = mace_mp(model=Potential_test, dispersion=True, default_dtype="float64", device='cuda')

computation = energy_computation(templates = cluster_template, go_conversion_rule = cluster_conversion_rule,   
                                 calculator = ase_calculator, 
                                 geo_opt_para = geo_opt_parameter, 
                                 calculator_type = 'ase',    
                                 if_coarse_calc = True, coarse_calc_para = coarse_opt_parameter,
                                 save_output_level = 'Simple',  
                                )

optimization = GA_ABC(computation.obj_func_compute_energy, cluster_boundary,
                      colony_size=10, limit=40, 
                      max_iteration=3, initial_population_scaler=2,
                      ga_interval=1, ga_parents=5, mutate_rate=0.5, mutat_sigma=0.05,
                      output_directory = 'results-5',
                      output_database = 'test-5.db'
                      )

Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.


C:\MyApp\Anaconda\envs\RDkit_from_globopt\Lib\site-packages\mace\calculators\mace.py:197: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, map_location

Using head default out of ['default']
Using TorchDFTD3Calculator for D3 dispersion corrections


In [ ]:
optimization.run(print_interval=1, if_return_results=False)

Setup ready...

        
     /\       _     _
    /  \     / \   / \       _ 
   /    \   /   \_/   \     / \   _
  /      \_/           \   /   \_/ \
 /                      \_/         \
/                                    \
████     ▄█▄    █   █   ███    ████   \
█   █    █ █    ██  █  █      █        \
█▄▄██   ██▄██   █ █ █  █  ██  ████      \
█  █    █   █   █  ██  █   █  █          \
█   █  █     █  █   █   ████   ████       \
                                           \___
----------------------------------------------->>
                                           
RANGE: a Robust Adaptive Nature-inspired Global Explorer of potential energy surfaces

Please cite: Difan Zhang, Małgorzata Z. Makoś, Roger Rousseau, Vassiliki-Alexandra Glezakou; RANGE: A robust adaptive nature-inspired global explorer of potential energy surfaces. J. Chem. Phys. 21 October 2025; 163 (15): 152501. https://doi.org/10.1063/5.0288910

For any questions, please contact us:
    Difan Zhang, Oak Ridge Na

C:\Users\d2j\OneDrive - Oak Ridge National Laboratory\Research_folder\Project_002_Glob_optimizer\RANGE\RANGE_go\energy_calculation.py:504: UserWarning: Gimbal lock detected. Setting third angle to zero since it is not possible to uniquely determine all angles.
  vec = self.cluster_to_vector( atoms, vec ) # update vec since we optimized the structure


In [ ]:
db = connect('test-5.db')
atoms = db.get_atoms(id=1) 
view( atoms, viewer='x3d' )
